In [ ]:
%matplotlib inline

In [ ]:
import sklearn #machine learning module: clustering, normalization, learning X->Y linear regression
import matplotlib #matlabs plotting library
import pandas as pd
import re
import subprocess
import glob
from tqdm import tqdm
import numpy as np 

In [ ]:
#change filepath to wherever you saved the .txt files for each of the 58 ca counties
infold='/PFAS-Datathon/'

In [ ]:
files=glob.glob(infold+'gama_*.txt')
len(files) #should be 58

In [ ]:
#takes the .txt files for all 58 counties and reformats them to have rows as sample ID and columns as chemical; saves these as .h5
for i in tqdm(range(len(files))):
    f=files[i]
    print(f)
    curdf=pd.read_csv(f,sep='\t',index_col=0,encoding='iso-8859-1').drop_duplicates().replace(0.0,np.nan)
    curdf=curdf.replace('UNK',0)
    numfilt=curdf['RL'].astype(float)<curdf['RESULTS'].astype(float)
    curdf=curdf[numfilt]
    sm=curdf[['LONGITUDE','LATITUDE']].drop_duplicates()
    qualifers=['=','nan','>',np.nan]
    qualifer_filt=curdf[curdf.QUALIFER.isin(qualifers)]
    qualifer_filt=qualifer_filt.drop_duplicates()
    qualifer_filt['CHEMICAL'] = qualifer_filt['CHEMICAL'].astype('category')
    qualifer_filt.index=[str(x)+';'+str(y) for x,y in zip(qualifer_filt.index,qualifer_filt['DATE'])]
    TMP=qualifer_filt['RESULTS'].groupby([qualifer_filt.index,qualifer_filt['CHEMICAL']])
    TMP=pd.DataFrame(TMP.max())
    w,c=zip(*[x for x in list(TMP.index)])
    TMP['well']=w
    TMP['chem']=c
    DFFFFFFFFFFFF=TMP.pivot(index='well',columns='chem',values='RESULTS')
    DFWELLMETA=qualifer_filt.iloc[:,6:]
    df_allsmetaresults=DFFFFFFFFFFFF.join(DFWELLMETA)
    unique=df_allsmetaresults.drop_duplicates()
    unique.to_hdf(infold+f.split('/')[-1].split('.')[0]+'.h5',key='PFAS')
    del curdf,sm,qualifer_filt,TMP,DFFFFFFFFFFFF,DFWELLMETA,df_allsmetaresults,unique

In [ ]:
files=glob.glob(infold+'gama_*.h5')
len(files) #check that it's still 58

In [ ]:
dfs=[]
for i in tqdm(range(len(files))):
    f=files[i]
    curdf=pd.read_hdf(f,key='PFAS')
    dfs.append(curdf)   

In [ ]:
#makes a dataframe including all 58 counties' data
DF=pd.concat(dfs)
DF

In [ ]:
#saves file
DF.to_hdf('filepath/filename.h5',key='bigfile')

In [ ]:
list(DF.columns)